# Librairies

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib 
import xgboost as xgb

# I. Importing Data and Cleaning Outliers

In [0]:
train_data = spark.read.csv("/mnt/data/train.csv", header=True, inferSchema=True)
test_data = spark.read.csv("/mnt/data/test.csv", header=True, inferSchema=True)
val_data = spark.read.csv("/mnt/data/val.csv", header=True, inferSchema=True)

# Combining train and test datasets
# We combine the two datasets to address the undersampling of label 'A'. The union operation merges the rows of both datasets.
data = train_data.union(test_data)

In [0]:
data = data.toPandas()

# On enlève valeurs aberrantes
data = data[data["Surface_habitable_logement"]<=100000]
data = data[data['Conso_5_usages/m²_é_finale']<=100000]
data = data[data['Hauteur_sous-plafond'] < 10]

display(data)

# II. Data Processing

In [0]:
numerical_data = data.select_dtypes(include=['int64', 'float64'])
categorical_data = data.select_dtypes(include=['object', 'category'])


In [0]:
# Count NA values
na_percentage = numerical_data.isna().mean().round(4)*100

print(na_percentage.sort_values())

In [0]:
cols_to_drop = na_percentage[na_percentage>10].index

data = data.drop(cols_to_drop, axis=1)
numerical_data = numerical_data.drop(cols_to_drop, axis=1)

data = data.drop(["Code_postal_(BAN)","Conso_5_usages_é_finale","Emission_GES_éclairage"], axis=1)
numerical_data = numerical_data.drop(["Code_postal_(BAN)","Conso_5_usages_é_finale","Emission_GES_éclairage"], axis=1)

for col in numerical_data :
    data[col] = data[col].fillna(data[col].mean())

In [0]:
# Count NA values
na_percentage_categorical = categorical_data.isna().mean().round(4)*100
print(na_percentage_categorical.sort_values())

In [0]:
cols_to_drop_cat = na_percentage_categorical[na_percentage_categorical>10].index
data = data.drop(cols_to_drop_cat, axis=1)
categorical_data = categorical_data.drop(cols_to_drop_cat, axis=1)

data = data.drop(["N°DPE","Code_INSEE_(BAN)","Qualité_isolation_plancher_bas","Nom__commune_(Brut)"], axis=1)
categorical_data = categorical_data.drop(["N°DPE","Code_INSEE_(BAN)","Qualité_isolation_plancher_bas","Nom__commune_(Brut)"], axis=1)

for col in categorical_data:
    data[col] = data[col].fillna(0)

In [0]:
print("Colonnes des variables numériques:")
for col in numerical_data.columns:
    print(col)
    
print("\n")

print("Colonnes des variables catégorielles:")
for col in categorical_data.columns:
    print(col)

In [0]:
data = data.drop(['_c0','Code_postal_(brut)'], axis=1)
data.dtypes

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

# III. Model XGBoost

In [0]:
data.columns

In [0]:
# Variables cibles et autres caractéristiques
y = data['Etiquette_DPE'].astype("category").cat.codes    
X = data.drop(['Etiquette_DPE'], axis=1)

# Encode les variables pour le modèle
for column in X.columns:
    X[column] = X[column].astype("category").cat.codes


print(" valeur de X :", X.columns)

In [0]:
display(y)

In [0]:
# Pensé à look si le  'Etiquette_DPE' est dans les colonnes
if 'Etiquette_DPE' in data.columns:
    # Variables cibles et autres caractéristiques
    y = data['Etiquette_DPE'].astype("category").cat.codes
    X = data.drop(['Etiquette_DPE'], axis=1)

    # Encode les variables pour le modèle
    for column in X.columns:
        X[column] = X[column].astype("category").cat.codes

    print( "valeurs de X :", X)
    print(" valeur de Y :", y)
    

    # Séparation des données
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Modèle d'arbre de décision
    model = xgb.XGBClassifier(num_class=7, n_estimators=100, learning_rate=0.1, max_depth=6)
    model.fit(X_train, y_train)

    # Évaluation
    y_pred = model.predict(X_test)
    train_accuracy = model.score(X_train, y_train)

    # Affichage de la précision de l'entraînement
    print("\n==== Résultats de l'entraînement ====")
    print(f"Précision sur les données d'entraînement : {train_accuracy:.2f}%")

    # Affichage de la précision de la validation
    print("\n==== Résultats de la validation ====")
    validation_accuracy = accuracy_score(y_test, y_pred)
    print(f"Précision sur les données de test : {validation_accuracy:.2f}%")

    # Affichage du rapport de classification
    print("\n==== Rapport de classification ====")
    print(classification_report(y_test, y_pred))

    # Affichage des comptages des valeurs réelles et prédites
    print("\n==== Comparaison des valeurs réelles et prédites ====")
    print("Comptage des valeurs réelles (y_test) :")
    print(y_test.value_counts())
    print("\nComptage des valeurs prédites (y_pred) :")
    print(pd.Series(y_pred).value_counts())
else:
    print("La colonne 'Etiquette_DPE' n'est pas dans les données.")


#  IV. Pred

In [0]:
val_data = val_data.toPandas()
val_data

In [0]:
val_data = val_data.drop(cols_to_drop, axis=1)
val_data = val_data.drop(cols_to_drop_cat, axis=1)
val_data = val_data.drop(["Code_INSEE_(BAN)", "Code_postal_(BAN)", "N°DPE","Conso_5_usages_é_finale","Qualité_isolation_plancher_bas","Code_postal_(brut)",
                          "Nom__commune_(Brut)","Emission_GES_éclairage"], axis=1)

print(val_data.columns)

In [0]:
# val_data = val_data.drop(cols_to_drop, axis=1)
# val_data = val_data.drop(cols_to_drop_cat, axis=1)
# val_data = val_data.drop(["Code_INSEE_(BAN)", "Code_postal_(BAN)", "N°DPE","Conso_5_usages_é_finale","Qualité_isolation_plancher_bas","Code_postal_(brut)",
#                           "Nom__commune_(Brut)","Emission_GES_éclairage"], axis=1)

for column in val_data.columns:
        val_data[column] = val_data[column].astype("category").cat.codes

prediction_val = model.predict(val_data)

In [0]:
prediction = pd.DataFrame(prediction_val, columns=['Etiquette_DPE'])
prediction['Etiquette_DPE'] = prediction['Etiquette_DPE'].astype('object')

# Remaplce les valeurs numériques par des lettres correspondant à l'étiquette DPE
mapping = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E", 5: "F", 6: "G"}
prediction['Etiquette_DPE'] = prediction['Etiquette_DPE'].replace(mapping)

# Affichez les fréquences des étiquettes DPE prédites
print("Fréquences des étiquettes DPE prédites:")
print(prediction['Etiquette_DPE'].value_counts())


df_soumission = pd.DataFrame()

val_data = spark.read.csv("/mnt/data/val.csv", header=True, inferSchema=True)
data_soumission = val_data.toPandas()

# verif si "N°DPE" est une colonne du CSV
if "N°DPE" in data_soumission.columns:
    df_soumission["N°DPE"] = data_soumission["N°DPE"]
    df_soumission["Etiquette_DPE"] = prediction['Etiquette_DPE']
else:
    print("La colonne 'N°DPE' n'est pas présente dans les données de validation.")


In [0]:
display(df_soumission)

In [0]:
df_soumission.to_csv("../Data/Prediction_model_metier_v1.csv", index=False)

# Interprétation

In [0]:
# Récupérer l'importance des caractéristiques
importances = model.feature_importances_

# Trier les caractéristiques par importance
indices = np.argsort(importances)[::-1]

# Afficher le classement des caractéristiques
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print(f"{f + 1}. feature {X_train.columns[indices[f]]} ({importances[indices[f]]:.4f})")

# Tracer l'importance des caractéristiques dans un graphique à barres horizontales
plt.figure(figsize=(10, 10))
plt.title("Feature importances")
bars = plt.barh(range(X_train.shape[1]), importances[indices], color="r", align="center")

# Ajouter des étiquettes pour chaque barre
for bar in bars:
    plt.text(
        bar.get_width(),  # Position x de l'étiquette
        bar.get_y() + bar.get_height() / 2,  # Position y de l'étiquette
        f'{bar.get_width():.2f}',  # Valeur de l'étiquette
        va='center',  # Centrer verticalement
        ha='left'  # Aligner horizontalement à gauche
    )

plt.yticks(range(X_train.shape[1]), X_train.columns[indices])
plt.gca().invert_yaxis()  # Inverser l'axe y
plt.xlabel('Importance')
plt.ylabel('Features')
plt.show()

In [0]:
for col in X_train.columns:
    print(col)